In [1]:
from tournament import tournament
from reporting import Reporting
import importlib
from collections import namedtuple
from game_agent_comp import CustomPlayerComp
from value_functions import improved_score_fast_x2,improved_score_fast,\
    improved_score_fast_x3, partition_score_x2
from sample_players import null_score
from policy import SimplePolicy
#importlib.reload(reporting)

In [29]:
import pickle, glob

files = glob.glob('result*.pickle')
print(files)

depths = {}
for file in files[:1]:
    try:
        with open(file, 'rb') as handle:
            old_depths = pickle.load(handle)
            #print(old_depths)
    except:
        old_depths={}

    #print(old_depths)
    for d in old_depths:
        for p, v in d.items():
            if p not in depths:
                depths[p]=v
            else:
                depths[p] = depths[p]+v

#with open('result.pickle', 'wb') as handle:
#    pickle.dump(depths, handle)
print(depths.keys())



['result0.pickle', 'result1.pickle', 'result2.pickle', 'result3.pickle', 'result4.pickle', 'result5.pickle', 'result6.pickle', 'result7.pickle', 'result8.pickle', 'result9.pickle']
dict_keys(['Faster improved', 'improved, two steps exact, with reporting', 'improved, three steps exact'])


In [31]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

example = depths['improved, three steps exact'][0]
move = example[1]
print(move)


{'depth': 6, 'score': 0.0, 'game': array([ 1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]), 'pos': (8, 15), 'simple_score': -7.0, 'move': (3, 2), 'allscores': [(-1.0, (0, 3)), (0.0, (3, 2)), (-1.0, (3, 0))]}


In [3]:
# collate all moves in one big list
moves = []
#for player, games in depths.items():
games=depths['improved, three steps exact']
for game in games:
    for move in game:
        if move['score'] != float('inf') and move['score'] != float('-inf'):
            moves.append(move)
print(len(moves))

1308


In [4]:
from neural import NNValueFunction
import numpy as np
from copy import copy
from sklearn.model_selection import train_test_split

train_moves, test_moves = train_test_split(moves, test_size = 0.2)


val = NNValueFunction([1,3,3])
val.set_coeff(np.random.normal(size =val.coeff_len)*0.3)
n = 0
train_diff = np.zeros(len(train_moves))
train_base = copy(train_diff)
test_diff = np.zeros(len(test_moves))
test_base = copy(test_diff)
while True:
    for m, move in  enumerate(train_moves):
        train_diff[m] = move['score'] - val(input_vec = move['game'], pos = move['pos'])
        train_base[m] = move['score']
        #square_print(nn(inp))
        #print(np.linalg.norm(diff))
        gr = val.nn.grad()
        gr = gr /(1 + np.linalg.norm(gr))
        coeff = val.nn.get_coeff() + gr.dot(train_diff[m]).transpose()[0]*0.001
        #print(coeff.shape)
        val.nn.set_coeff(coeff)
        #if m%100 == 0
    for m,move in enumerate(test_moves):
        test_diff[m] = move['score'] - val(input_vec = move['game'], pos = move['pos'])
        test_base[m] = move['score']
    print('train: ',np.linalg.norm(train_diff)/np.linalg.norm(train_base))
    print('test: ',np.linalg.norm(test_diff)/np.linalg.norm(test_base))


1 797
train:  1.00814822617
test:  1.00404461153
train:  1.00416601026
test:  1.00170380924
train:  1.00135766159
test:  1.00004442707
train:  0.999230749539
test:  0.998745976651
train:  0.997431032404
test:  0.997608809176
train:  0.995762434587
test:  0.996539476944
train:  0.994153261169
test:  0.995512990708
train:  0.992600675791
test:  0.994534688396
train:  0.991125650984
test:  0.99361791433
train:  0.989749252526
test:  0.992772052133
train:  0.988482080531
test:  0.991999946612
train:  0.987325791058
test:  0.99130000372
train:  0.98627623065
test:  0.990667894385
train:  0.985325958871
test:  0.990097948667
train:  0.984466129345
test:  0.989584067596
train:  0.983687681217
test:  0.989120288062
train:  0.982981865104
test:  0.988700988822
train:  0.982340591441
test:  0.988321056244
train:  0.981756570248
test:  0.987975937252
train:  0.98122332679
test:  0.987661606388
train:  0.980735122572
test:  0.987374511988
train:  0.980286927063
test:  0.987111564634
train:  0.9798

KeyboardInterrupt: 

In [ ]:
def transpose_list_of_lists(lol):
    
    transp = []

    for game in lol:
        for m, move_depth in enumerate(game):
            if len(transp) <= m:
                transp.append([])
            transp[m].append(move_depth)
    return transp



In [ ]:
#%pdb
from statistics import mean, stdev, median
import numpy as np

def get_mean_std(lol):
    result = {'mu':[], 'std':[],'median':[], 'strict_win':[], 'strict_loss':[]}
    
    for move in lol:
        if len(move) > 2:
            result['mu'].append(mean([m['depth'] for m in move]))
            result['std'].append(stdev([m['depth'] for m in move]))
            result['median'].append(median([m['depth'] for m in move]))
            result['strict_win'].append(\
                len([ m['score'] for m in move if m['score'] == float('inf')]) / len(move))
            result['strict_loss'].append(\
                len([ m['score'] for m in move if m['score'] == float('-inf')]) / len(move))
            
            
    result['mu'] = np.array(result['mu'] )
    result['std'] = np.array(result['std'])
    result['median'] = np.array(result['median'])
    result['strict_win'] = np.array(result['strict_win'])
    result['strict_loss'] = np.array(result['strict_loss'])
    return result

player_stats = {}
for p, d in depths.items():
    tmp = transpose_list_of_lists(d)
    #print(tmp)
    player_stats[p] = get_mean_std(transpose_list_of_lists(d))


In [ ]:
player_stats


In [ ]:
#import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
ax = plt.axes()
#ax.set_yscale("log")
lines = []
legend = []
n=0
lastpt = 12
for p, stats in player_stats.items():
    lines.append(\
        plt.errorbar(np.array(range(lastpt))+n,stats['mu'][0:lastpt], 
                     yerr=stats['std'][0:lastpt], fmt='o'))
    legend.append(p)
    n+= 0.1

plt.legend(lines,legend)
plt.show()

In [ ]:
plt.figure()
ax = plt.axes()
#ax.set_yscale("log")
lines = []
legend =[]
for p, stats in player_stats.items():
    lines.append(\
        plt.plot(range(len(stats['mu'])),stats['strict_win'] + stats['strict_loss']))
    legend.append(p)
plt.legend(legend)
plt.grid()
plt.show()

In [ ]:
plt.figure()
ax = plt.axes()
#ax.set_yscale("log")
lines = []
legend =[]
for p, stats in player_stats.items():
    lines.append(\
        plt.plot(range(len(stats['mu'])),stats['strict_win']-stats['strict_loss']))
    legend.append(p)
plt.legend(legend)
plt.grid()
plt.show()